## Import modules and create spark session

In [20]:
#import modules
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

#create Spark session
appName = "Sentiment Analysis in Spark"
conf = (SparkConf().setAppName("appName"))

sc = SparkContext.getOrCreate()
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

## Read data file into Spark dataFrame

In [6]:
#read csv file into dataFrame with automatically inferred schema
schema = StructType([
    StructField("Target", IntegerType(), True),
    StructField("ID", IntegerType(), True),
    StructField("Date", StringType(), True),
    StructField("Query", StringType(), True),
    StructField("User", StringType(), True),
    StructField("Text", StringType(), True)])

tweets_csv = spark.read.csv('dataset/tweets.csv', schema=schema, header=False)
tweets_csv.show(truncate=False, n=3)

+------+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|Target|ID        |Date                        |Query   |User           |Text                                                                                                               |
+------+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|0     |1467810369|Mon Apr 06 22:19:45 PDT 2009|NO_QUERY|_TheSpecialOne_|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|
|0     |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY|scotthamilton  |is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |
|0     |1467810917|Mon Apr 06 22:19:53 PDT 2009|NO

## Select the related data

In [7]:
#select only "SentimentText" and "Sentiment" column, 
#and cast "Sentiment" column data into integer
data = tweets_csv.select("Text", col("Target").alias("label"))
data.show(truncate = False,n=5)

+-------------------------------------------------------------------------------------------------------------------+-----+
|Text                                                                                                               |label|
+-------------------------------------------------------------------------------------------------------------------+-----+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|0    |
|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |0    |
|@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                          |0    |
|my whole body feels itchy and like its on fire                                                                     |0    |
|@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.     |0    |
+-------

## Trim White Space 

In [8]:
from pyspark.sql.functions import trim
data = data.withColumn("Text", trim(data.Text))
data.show(truncate=False, n=5)

+-------------------------------------------------------------------------------------------------------------------+-----+
|Text                                                                                                               |label|
+-------------------------------------------------------------------------------------------------------------------+-----+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|0    |
|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |0    |
|@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                          |0    |
|my whole body feels itchy and like its on fire                                                                     |0    |
|@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.     |0    |
+-------

## Divide data into training and testing data

In [9]:
#divide data, 70% for training, 30% for testing
dividedData = data.randomSplit([0.7, 0.3]) 
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 1120055 ; Testing data rows: 479945


## Prepare training data

Separate "SentimentText" into individual words using tokenizer

In [10]:
tokenizer = Tokenizer(inputCol="Text", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False, n=5)

+--------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Text                                                                                                                                              |label|SentimentWords                                                                                                                                                              |
+--------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|!!!!!!!!!! OMG!

Removing stop words (unimportant words to be features)

In [11]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

+--------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|Text                                                                                                                                              |label|SentimentWords                                                                                                                                                              |MeaningfulWords                                                                                                       |
+---------------------------------------------------------------------------------------------------------

Converting words feature into numerical feature. In Spark 2.2.1,it is implemented in HashingTF funtion using Austin Appleby's MurmurHash 3 algorithm

In [12]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=False, n=3)

+-----+------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|MeaningfulWords                                                                                                   |features                                                                                                                                                        |
+-----+------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[!!!!!!!!!!, omg!!!!!!!, lost, photage, stuff, new, video, coming, monday..., sorry, guys, looks, like, new, vid] |(262144,[12409,19153,70864

## Train our classifier model using training data

In [21]:
nb = NaiveBayes(smoothing=1)
model = nb.fit(numericTrainData)
print ("Training is done!")

Training is done!


## Save the Model

In [23]:
basePath = "/home/mecha/Documents/ml_models/navies_bayes_sentiment_analyzer"
model.save(basePath + "/modeliter10")


23/01/18 13:43:17 WARN TaskSetManager: Stage 42 contains a task of very large size (4185 KiB). The maximum recommended task size is 1000 KiB.


## Prepare testing data

In [24]:
tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)


+-----+--------------------------------------------------------------+-------------------------------------------------------------------------------------------------+
|Label|MeaningfulWords                                               |features                                                                                         |
+-----+--------------------------------------------------------------+-------------------------------------------------------------------------------------------------+
|0    |[!!, @jordaniscreativ, ...nice, seeing, out!, bad, get, chat!]|(262144,[22593,82593,93729,130329,145380,153094,244891,252722],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|0    |[!!, de, creer, , , , , http://bit.ly/ln2da]                  |(262144,[43265,93729,162985,181897,249180],[1.0,1.0,1.0,1.0,4.0])                                |
+-----+--------------------------------------------------------------+-------------------------------------------------------------------------------------

## Predict testing data and calculate the accuracy model

In [25]:
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
    "MeaningfulWords", "prediction", "Label")
predictionFinal.show(n=4, truncate = False)
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("correct prediction:", correctPrediction, ", total data:", totalData, 
      ", accuracy:", correctPrediction/totalData)

23/01/18 13:43:30 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


+-----------------------------------------------------------------------------------------------------------------+----------+-----+
|MeaningfulWords                                                                                                  |prediction|Label|
+-----------------------------------------------------------------------------------------------------------------+----------+-----+
|[!!, @jordaniscreativ, ...nice, seeing, out!, bad, get, chat!]                                                   |1.0       |0    |
|[!!, de, creer, , , , , http://bit.ly/ln2da]                                                                     |1.0       |0    |
|[!!!, , awwee, damnnn., sorry, joshyy,, wish, could,, work, like, day..., lunch, supper, shift., @joshalexanderr]|0.0       |0    |
|[!!!, , left, teddy, bear, kim, &amp;, scott's.]                                                                 |1.0       |0    |
+--------------------------------------------------------------------

correct prediction: 184913 , total data: 479945 , accuracy: 0.3852795632832929
